# MMVCの学習に必要なconfig系Fileを作成する

ver.2022/07/02



## 1 概要
「Google Colaboratory」を利用した、MMVCの学習を実行するために必要なファイルを生成します。

## 2 Google Drive をマウント
**Google Drive にアップロードした MMVC_Trainer を参照できるように、設定します。**

「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」

といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

成功すれば、下記メッセージが出ます。
```
Mounted at /content/drive/
```


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

cdコマンドを実行して、マウントしたGoogle Drive のMMVC_Trainerディレクトリに移動します。

%cd 「MMVC_Trainerをgoogle driveにパップロードしたパス」

としてください。

正しいパスが指定されていれば

-rw------- 1 root root 11780 Mar  4 16:53 attentions.py

-rw------- 1 root root  4778 Mar  4 16:53 commons.py

drwx------ 2 root root  4096 Mar  5 15:20 configs

...といった感じに表示されるはずです。

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/debug/MMVC_Trainer-1.2.2.0-flow4
!ls -la

## 3 必要なライブラリのインストール

何も考えず実行してください。

In [ ]:
!pip install pyopenjtalk
!pip install torch==1.8.0
!pip install Unidecode==1.1.1
!pip install phonemizer==2.2.1
!pip install retry
!pip install librosa==0.8.0
!pip install torchaudio==0.8.0
!apt install sox

#4 自分の声の音声正規化

#5 学習済みの音声から自分の声ににた話者を探す 
学習済みの話者からあなたの声に似た話者を探します。  
**1時間程度**かかるのでゆっくりお待ちください。

In [ ]:
!python most_likely_voice.py -f "fine_model/G_62000.pth"

## 5 MMVCの学習に必要なconfig系Fileを作成する

学習に必要なファイルを実行します。  
オプションは-f -s -c -zがあります。普通に使う際には-cは不要です。  
-f 作成するconfig系Fileの名前です。よくわからない人は変更不要です。  
-s サンプリングレートです。datasetの音声のサンプリングレートに合わせて指定ください。よくわからない人は変更不要です。  
-z ずんだもんフラグです。ずんだもんの学習を行うか行わないかTrue or False で指定します。セルを分けているので使い分けてください。  

実行時に  
「WARNING: JPCommonLabel_insert_pause() in jpcommon_label.c: First mora should not be short pause.」  
と表示されますが、無視して大丈夫です。 

「filelists」ディレクトリに  
「指定したconfig系Fileの名前_textful.txt」  
「指定したconfig系Fileの名前_textful_val.txt」  
「指定したconfig系Fileの名前_textless.txt」  
「指定したconfig系Fileの名前_val_textless.txt」  
「指定したconfig系Fileの名前_Correspondence.txt」  
上記5ファイル

「configs」ディレクトリに  
「指定したconfig系Fileの名前.json」  
上記1ファイルが生成されていれば、成功です。 



**※以下興味がある方向け**
create_dataset_jtalk.pyではベースになるjsonファイルを書き換えています。  
デフォルトでは、「/configs/baseconfig.json」が指定されており、このファイルの  
「training_files」「validation_files」「training_files_notext」「validation_files_notext」「sampling_rate」「n_speakers」  
の6項目を書き換えています。  
-c オプションではベースになるjsonファイルのpathを指定することができます。   
デフォルトのハイパーパラメータを変更した場合等に利用ください。  
開発者は一度も使ったことはありません。  

In [ ]:
!python create_dataset_jtalk.py -f train_config_zundamon -s 24000 -m 69 -z True

ずんだもん以外の話者の学習を行う場合、下記セルを実行してください。

In [ ]:
!python create_dataset_jtalk.py -f train_config -m 69 -s 24000

自分とずんだもんの音声データの音量を揃える

In [ ]:
!mkdir dataset/textful/00_myvoice/wav/normalize
!for wav in dataset/textful/00_myvoice/wav/*.wav ; do DB=`sox $wav -n -S stats |& grep "RMS Pk dB"`; echo $wav $DB | perl -ane '$stddb=-19.55; $db=$F[4]; $gain=$stddb-$db; $path=$F[0]; $path=~/^(.*\/)(.*\.wav)$/; $dir=$1; $file=$2; system "sox ${dir}${file} ${dir}normalize/${file} gain $gain\n"' ; done
!mv dataset/textful/00_myvoice/wav dataset/textful/00_myvoice/orig; mv dataset/textful/00_myvoice/orig/normalize dataset/textful/00_myvoice/wav

In [ ]:
!mkdir dataset/textful/1205_zundamon/wav/normalize
!for wav in dataset/textful/1205_zundamon/wav/*.wav ; do DB=`sox $wav -n -S stats |& grep "RMS Pk dB"`; echo $wav $DB | perl -ane '$stddb=-19.55; $db=$F[4]; $gain=$stddb-$db; $path=$F[0]; $path=~/^(.*\/)(.*\.wav)$/; $dir=$1; $file=$2; system "sox ${dir}${file} ${dir}normalize/${file} gain $gain\n"' ; done
!mv dataset/textful/1205_zundamon/wav dataset/textful/1205_zundamon/orig; mv dataset/textful/1205_zundamon/orig/normalize dataset/textful/1205_zundamon/wav